<a href="https://colab.research.google.com/github/RiccardoRubini93/SWA_DQ/blob/main/2505_ws_raw_to_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
pip install gcsfs

In [15]:
import pandas as pd
import re

In [16]:
def check_for_valid_url(url: str) -> bool:
    """checks if url is valid"""
    regex = re.compile(
        r"^(?:http|ftp)s?://"  # http:// or https://
        r"(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|"  # domain...
        r"localhost|"  # localhost...
        r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})"  # ...or ip
        r"(?::\d+)?"  # optional port
        r"(?:/?|[/?]\S+)$",
        re.IGNORECASE,
    )
    return re.match(regex, url) is not None

In [21]:
brand = 'pandora_us'
params = {}
params['raw_bucket'] = 'swarovski-advanced-analytics-data-ingestion-prod'
params['raw_prefix'] = 'raw/25_webscraper_io/2505/v1/2023'
params['bq_project'] = "advanced-analytics-278408"
params['bq_dataset'] = "conf__competitorbasket"

In [10]:
path = f"gs://{params['raw_bucket']}/{params['raw_prefix']}/{dstring[:4]}/2505__cb__{brand}__ww__{ds}.csv"

In [12]:
ds = '2023-04-01'
dstring = ds
dstring_nodash = dstring.replace("-", "")
file_df = pd.read_csv('gs://swarovski-advanced-analytics-data-ingestion-prod/raw/25_webscraper_io/2505/v1/2023/2505__cb__pandora_us__ww__2023-04-01.csv')
# Strip all string columns
file_obj = file_df.select_dtypes(['object'])
file_df[file_obj.columns] = file_obj.apply(lambda x: x.str.strip())

In [13]:
file_df

,product_url,country,sku,price
0,https://us.pandora.net/en/earrings/earrings/ho...,us,286317C01,100.0
1,https://us.pandora.net/en/rings/rings/promise-...,us,196316CZ,45.0
2,https://us.pandora.net/en/charms-bracelets/bra...,us,598400C00,65.0
3,https://us.pandora.net/en/necklaces-pendants/n...,us,359520C01,595.0
4,https://us.pandora.net/en/earrings/earrings/ho...,us,296317CZ,65.0
5,https://us.pandora.net/en/rings/rings/band-rin...,us,160050C01,115.0
6,https://us.pandora.net/en/rings/rings/promise-...,us,190050C01,95.0
7,https://us.pandora.net/en/charms-bracelets/bra...,us,590719,65.0
8,https://us.pandora.net/en/necklaces-pendants/n...,us,399685C00,125.0
9,https://us.pandora.net/en/earrings/earrings/ho...,us,266317C01,100.0


In [19]:
from google.cloud import bigquery

In [22]:
scraper_name = 'pandora_us'
table_name, country_name = scraper_name.rsplit("_", 1)
bq_client = bigquery.Client(project="advanced-analytics-278408")
query = f"SELECT sku, url_{country_name} FROM `{params['bq_project']}.{params['bq_dataset']}.cb__{table_name}`"
query_job = bq_client.query(query)
query_df = query_job.result().to_dataframe()
query_df = query_df[query_df[f'url_{country_name}'].apply(check_for_valid_url)]

In [27]:
scraper_name.rsplit("_", 1)

['pandora', 'us']

In [25]:
if not query_df["sku"].str.strip().isin(file_df["sku"].str.strip()).all():
        # fail airflow dag and print out the missing product_skus
            #raise AirflowFailException("Missing product_skus: {}".format(query_df["sku"][~query_df["sku"].isin(file_df["sku"].str.strip())]))
    print("Missing product_skus: {}".format(query_df["sku"][~query_df["sku"].isin(file_df["sku"].str.strip())]))

        # check for empty rows in price column in dior_df
if file_df["price"].isnull().any():
            #raise ValueError("The following rows in price column are empty: {}".format(file_df[file_df["price"].isnull()]))
    print("The following rows in price column are empty: {}".format(file_df[file_df["price"].isnull()]))

Missing product_skus: 0    590702HV
Name: sku, dtype: object


In [32]:
# everything here clear the currency formatting
def cleanse_price(x):
    x = re.sub(r"(\.\d{2})(?!\d)|(\,\d{2})(?!\d)", "", x)
    x = re.sub(r"\D", "", x)
    return x

file_df["price"] = file_df["price"] #.apply(cleanse_price)
        
# write to bigquery table
file_df["extraction-date"] = dstring
file_df["brand"] = scraper_name.rsplit("_", 1)[0]

#define column order 
cols_order = ['product_url','country','sku','price','extraction-date','brand']
file_df = file_df[cols_order]

In [36]:
file_df

,product_url,country,sku,price,extraction-date,brand
0,https://us.pandora.net/en/earrings/earrings/ho...,us,286317C01,100.0,2023-04-01,pandora
1,https://us.pandora.net/en/rings/rings/promise-...,us,196316CZ,45.0,2023-04-01,pandora
2,https://us.pandora.net/en/charms-bracelets/bra...,us,598400C00,65.0,2023-04-01,pandora
3,https://us.pandora.net/en/necklaces-pendants/n...,us,359520C01,595.0,2023-04-01,pandora
4,https://us.pandora.net/en/earrings/earrings/ho...,us,296317CZ,65.0,2023-04-01,pandora
5,https://us.pandora.net/en/rings/rings/band-rin...,us,160050C01,115.0,2023-04-01,pandora
6,https://us.pandora.net/en/rings/rings/promise-...,us,190050C01,95.0,2023-04-01,pandora
7,https://us.pandora.net/en/charms-bracelets/bra...,us,590719,65.0,2023-04-01,pandora
8,https://us.pandora.net/en/necklaces-pendants/n...,us,399685C00,125.0,2023-04-01,pandora
9,https://us.pandora.net/en/earrings/earrings/ho...,us,266317C01,100.0,2023-04-01,pandora
